In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## מיזוג וחיבור טבלאות (Merging & Joining)

בעת ניתוח נתונים ניסיוניים, לעיתים המידע מפוצל בין כמה טבלאות:  
טבלת **מדידות** (למשל קריאות מתח/זרם), טבלת **מטא־דאטה של ניסויים** (תאריך, קונפיגורציה), וטבלת **חיישנים/חומרים** (כיול, חומר הדגימה).  
כדי לקבל תמונה מלאה, נבצע מיזוגים (join/merge) לפי מפתחות משותפים.

#### סוגי מיזוגים (merge types)
- **inner** — מחזיר רק רשומות עם התאמה בשתי הטבלאות.  
- **left** — שומר את כל שורות הטבלה השמאלית, ומשלים נתונים מהימנית אם נמצאה התאמה.  
- **right** — אנלוגי ל־left, אך שומר את כל הימנית.  
- **outer** — איחוד מלא: כל השורות משתי הטבלאות, עם NaN כאשר אין התאמה.

#### בחירת מפתחות ו־suffixes
בעת מיזוג, נגדיר את שם/שמות העמודות לשיוך דרך `on=` או `left_on`/`right_on`.  
כאשר יש שמות עמודות חופפים שאינם מפתחות, נשתמש ב־`suffixes=("_meas", "_meta")` כדי להבדיל בין המקורות.

#### מלכודות נפוצות (Join Pitfalls)
- **מפתחות כפולים → התפוצצות שורות**: אם לטבלה אחת יש כמה שורות עם אותו מפתח, ולטבלה השנייה גם — המיזוג יוצר קרוס־מכפלות.  
- **ולידציה של יחסי צימוד**: אפשר (וכדאי) להשתמש ב־`validate=` כדי לאכוף יחס קישור צפוי:
  - `'m:1'` — רבים־לאחד (למשל כמה מדידות לאותו ניסוי).
  - `'1:1'` — אחד־לאחד (מזהה ייחודי בשתי הטבלאות).
  - `'1:m'` — אחד־לרבים, וכדומה.

### תרגילים: Merging & Joining בנתונים פיסיקליים
נבצע מיזוג בין **מדידות** לבין **מטא־דאטה של ניסוי** (left join),  
נוסיף טבלת **חיישנים** לחישוב **מרווח בטיחות (margin)** מול סף כיול,  
נשתמש ב־`validate='m:1'` כדי לוודא תקינות, ונזהה/נתקן כפילויות מפתחות לפני המיזוג.

#### תרגיל 1: מיזוג נתוני מדידות עם מידע מטא־דאטה של ניסויים

בתרגיל זה נלמד כיצד לבצע **מיזוג (Join)** בין שתי טבלאות — אחת המכילה **נתוני מדידות חשמליות**, והשנייה המכילה **מידע כללי על הניסויים** (מטא־דאטה).  
התרגיל מדגים מקרה קלאסי בניתוח נתונים פיזיקליים, שבו נרצה לשלב את תוצאות הניסוי עם פרטי התנאים שבהם בוצע.

שלבי העבודה:
1. **יצירת טבלת מדידות (`measurements`)**  
   כל שורה מייצגת מדידה אחת, הכוללת מזהה מדידה (`measurement_id`), מזהה ניסוי (`experiment_id`), מדידות של **מתח** ו־**זרם**, וזמן המדידה (`timestamp`).  
   נשים לב כי לכל ניסוי עשויות להיות מספר מדידות — כלומר מדובר ביחס **רבים־לאחד** (m:1).

2. **יצירת טבלת מטא־דאטה (`experiments`)**  
   כאן יש שורה אחת לכל ניסוי, עם מידע כללי: תצורת הניסוי (`config`), שם המפעיל (`operator`), וטמפרטורת החדר בזמן הניסוי (`room_temp_C`).

3. **ביצוע מיזוג (Left Join)**  
   נשתמש בפונקציה `merge()` של Pandas עם הפרמטרים:
   - `on="experiment_id"` — כדי למזג לפי מזהה הניסוי המשותף לשתי הטבלאות.  
   - `how="left"` — כדי לשמור את כל המדידות גם אם אין להן התאמה במטא־דאטה.  
   - `validate="m:1"` — כדי לוודא שכל ניסוי מופיע בדיוק פעם אחת בצד המטא־דאטה, וכך למנוע כפילויות.

4. **תוצאה**  
   נקבל טבלה מאוחדת שבה לכל מדידה נוספו הנתונים המתאימים על הניסוי שממנו נלקחה — מה שמאפשר לנתח את התוצאות תוך התחשבות בתנאים הפיזיקליים.

תרגיל זה מהווה שלב חשוב בכל ניתוח ניסויי: שילוב מדידות גולמיות עם נתוני רקע מאפשר להבין כיצד תנאים שונים משפיעים על התוצאות.



In [ ]:
# --- Exercise 1: Left join of measurements with experiment metadata ---
import pandas as pd

# Measurements table: multiple measurements per experiment_id
measurements = pd.DataFrame({
    "measurement_id": [101, 102, 103, 104, 105],
    "experiment_id":  ["EXP1", "EXP1", "EXP2", "EXP3", "EXP3"],
    "voltage":        [5.0,   4.8,   12.0,  3.2,   3.3],
    "current":        [0.20,  0.18,  0.55,  0.10,  0.11],  # Amperes
    "timestamp": pd.to_datetime([
        "2025-01-03 10:00", "2025-01-03 10:05",
        "2025-01-05 14:20", "2025-01-06 09:10", "2025-01-06 09:12"
    ])
})

# Experiment metadata: one row per experiment_id (intended)
experiments = pd.DataFrame({
    "experiment_id": ["EXP1", "EXP2", "EXP3"],
    "config":        ["A",    "B",    "C"],
    "operator":      ["Dana", "Noam", "Rivka"],
    "room_temp_C":   [22.3,   21.7,   23.1]
})

# Left join: keep all measurements, add metadata where available
merged_1 = measurements.merge(
    experiments,
    on="experiment_id",
    how="left",
    validate="m:1"  # many measurements to one experiment metadata
)

print("Merged measurements with experiment metadata (left join):")
display(merged_1)


#### תרגיל 2: הוספת טבלת חיישנים וחישוב מרווח בטיחות (Margin)
נוסיף טבלת **חיישנים** הכוללת סף ייחוס (threshold) להספק.  
נחשב הספק לכל מדידה \( P = V \cdot I \) ונגדיר **מרווח בטיחות** כ־`margin = P - threshold`.  
נשתמש ב־`suffixes` כדי להבדיל בין עמודות חופפות, ונאכוף יחס `'m:1'`.


In [ ]:
# --- Exercise 2: Add sensors and compute power margins; validate 'm:1' ---
# Sensors table: one row per config (intended), with a calibration threshold for power (Watts)
sensors = pd.DataFrame({
    "config":    ["A",  "B",  "C"],
    "threshold": [0.90, 6.00, 0.40]   # Power threshold per configuration (W)
})

# Merge previous result with sensors by 'config'
merged_2 = merged_1.merge(
    sensors,
    on="config",
    how="left",
    validate="m:1",              # many measurements per single config row
    suffixes=("", "_sensor")
)

# Compute power and margin
merged_2["power_W"] = merged_2["voltage"] * merged_2["current"]
merged_2["margin_W"] = merged_2["power_W"] - merged_2["threshold"]

print("Merged with sensors and computed margins:")
display(merged_2[[
    "measurement_id", "experiment_id", "config", "voltage", "current",
    "power_W", "threshold", "margin_W"
]])


#### תרגיל 3: זיהוי ותיקון מפתחות כפולים לפני מיזוג
לעיתים בטבלאות המטא־דאטה או החיישנים מופיעות **כפילויות מפתח** (למשל כמה שורות עם אותו `config`).  
אם נמזג כך, נקבל **התפוצצות שורות**.  
נזהה כפילויות, נתקן (למשל איגום/סינון) ואז נבצע מיזוג בטוח.


In [ ]:
# --- Exercise 3: Identify and fix duplicate keys before merging ---
# Introduce a duplicate on purpose: sensors_dup has two rows for config 'B'
sensors_dup = pd.DataFrame({
    "config":    ["A",  "B",  "B",  "C"],
    "threshold": [0.90, 6.00, 6.10, 0.40]
})

# 1) Detect duplicate keys
dups = sensors_dup["config"].duplicated(keep=False)
print("Duplicate keys in sensors_dup:")
display(sensors_dup[dups])

# 2) Fix duplicates by aggregating to a single threshold per config (e.g., take mean or max)
#    Here we choose the stricter threshold by taking the max.
sensors_fixed = (sensors_dup
                 .groupby("config", as_index=False, sort=False)
                 .agg({"threshold": "max"}))

print("Sensors after fixing duplicates (aggregated by max threshold):")
display(sensors_fixed)

# 3) Safe merge with validation (now each config is unique)
merged_safe = merged_1.merge(
    sensors_fixed,
    on="config",
    how="left",
    validate="m:1"
)

# Compute power & margin again using fixed thresholds
merged_safe["power_W"] = merged_safe["voltage"] * merged_safe["current"]
merged_safe["margin_W"] = merged_safe["power_W"] - merged_safe["threshold"]

print("Merged safely after fixing duplicates:")
display(merged_safe[[
    "measurement_id", "experiment_id", "config", "power_W", "threshold", "margin_W"
]])


#### תרגיל 4: אימות ספירת שורות (Row Count Assertions)
לאחר מיזוגים, חשוב לוודא שספירת השורות תואמת לציפיות.  
לדוגמה, במיזוג **רבים־לאחד** (m:1) לא אמורה להיות **עלייה** במספר השורות.  
נבצע בדיקות פשוטות: התאמת מספר השורות לפני ואחרי המיזוג, ושימוש בפרמטר `validate` כקו הגנה ראשון.


In [ ]:
# --- Exercise 4: Assert row counts and use validate as guardrail ---
left_rows = len(merged_1)     # after first left-join (measurements + experiments)
safe_rows = len(merged_safe)  # after merging fixed sensors

print(f"Row count after experiments merge: {left_rows}")
print(f"Row count after sensors merge (fixed): {safe_rows}")

if safe_rows != left_rows:
    raise AssertionError("Row count changed unexpectedly after an m:1 merge!")

print("✅ Row count preserved. Validation and duplicate handling succeeded.")


`````{admonition} סיכום
:class: tip
בפרק זה למדנו כיצד לבצע **מיזוגים (Merging & Joining)** בצורה נכונה ובטוחה בניתוח נתונים ניסיוניים:

- היכרנו את סוגי המיזוגים העיקריים (`inner`, `left`, `right`, `outer`) ואת השימוש הנפוץ שלהם במצבי **איחוד נתוני מדידות, מטא־דאטה וחיישנים**.  
- הבנו כיצד לבחור **מפתחות (keys)** למיזוג, להשתמש ב־`suffixes` כדי להבדיל בין עמודות זהות, ולוודא **יחסי צימוד תקינים** בעזרת הפרמטר `validate` (כגון `'m:1'` או `'1:1'`).  
- ראינו כיצד מפתחות כפולים עלולים לגרום ל־**התפוצצות שורות (row explosion)**, וכיצד לזהות ולתקן כפילויות לפני מיזוג — למשל באמצעות איגום (`groupby`) או סינון.  
- למדנו לחשב ערכים חדשים לאחר המיזוג, כמו **מרווח בטיחות (margin)** בין ההספק הנמדד לסף הכיול, ולהשתמש ב־assertions לאימות תקינות הנתונים.  

באמצעות כלים אלו ניתן לבנות מערכי נתונים מאוחדים ומדויקים —  
שמהם ניתן להמשיך לנתח, לחשב אנרגיות כוללות, לבצע השוואות בין ניסויים שונים,  
ולבסוף להבטיח שהתובנות מבוססות על נתונים נקיים, עקביים ומהימנים.
`````

In [ ]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "בעת עיבוד נתוני ניסוי במעבדה, יש ברשותך טבלת מדידות (measurement_id, experiment_id, voltage, current) וטבלת מטא־דאטה של ניסויים (experiment_id, operator, config).<br><br>כיצד תבצע מיזוג נכון שיכלול את כל המדידות, יוודא שאין כפילויות בטבלת המטא־דאטה, וישמור על מספר השורות המקורי?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "ביצוע merge עם how='left' על experiment_id, תוך שימוש בפרמטר validate='m:1' לוודא שכל ניסוי מופיע פעם אחת בלבד בטבלת המטא־דאטה.",
      "correct": true,
      "feedback": "נכון! מיזוג left מבטיח שכל המדידות נשמרות, ו־validate='m:1' מגן מפני כפילויות בצד המטא־דאטה."
    },
    {
      "answer": "ביצוע merge עם how='inner' כדי לכלול רק רשומות משותפות, ללא צורך ב־validate.",
      "correct": false,
      "feedback": "לא — מיזוג inner יגרום לאובדן מדידות שלא נמצאות בטבלת המטא־דאטה, ואינו מבטיח תקינות יחסים."
    },
    {
      "answer": "שימוש ב־concat כדי לחבר את שתי הטבלאות זו על גבי זו, ולבדוק כפילויות לאחר מכן.",
      "correct": false,
      "feedback": "לא — concat מחבר שורות, לא עמודות. כאן נדרש מיזוג לפי מפתח משותף, לא איחוד אנכי."
    },
    {
      "answer": "ביצוע merge ללא ציון validate כדי להימנע משגיאות בזמן הריצה.",
      "correct": false,
      "feedback": "לא מומלץ — ללא validate עלולות להיווצר כפילויות או שורות מיותרות שלא תזוהינה."
    }
  ]
}]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)
